- [x] histogram of final vote classes (overall and per university)
- [ ] box plot of test scores per university
- [ ] interrator score

General tasks:
- [x] collate labeled files in one dataframe
- [ ] collate and preprocess (if necessary) test scores
- [ ] provide basic quali and quanti analyses

seaborn



## To check
- dapat walang "VOTE" under final act
- delete other columns

In [33]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import glob

In [34]:
data_dir = 'data/'
data_raw_dir = data_dir +'raw/'
data_proc_dir = data_dir +'proc/'

data_raw_label_dir = data_raw_dir + 'label/'
data_raw_test_dir = data_raw_dir + 'test/'

In [35]:
# merge all excels into 1 data frame
label_df = pd.DataFrame()
column_names = ['text', 'person', 'translation', 'a1', 'a2', 'match', 'final_act', 'timestamp', 'school', 'filename', 'student']
column_name_change = {'TEXT': 'text', 'ORIGINAL TEXT/CONVO': 'text', 'Original text': 'text', 'PERSON': 'person',
                      'TRANSLATION': 'translation', 'A1': 'a1',
                      'A2': 'a2', 'Match': 'match', 'MATCH': 'match',
                      'FINAL': 'final_act', 'Final Act': 'final_act', 'Timestamp': 'timestamp', 'Student': 'student'}

In [36]:
# iterate folders
for folder in glob.iglob(data_raw_label_dir + '*/'):
    for filename in glob.iglob(folder +'*.xlsx'):
        school_name = folder.replace(data_raw_label_dir, '').replace('/', '')
        filename = filename.replace(folder, '')
        temp_df = pd.read_excel(data_raw_label_dir + school_name + '/' + filename)
        temp_df = temp_df.rename(columns= column_name_change)

        # add additional columns
        temp_df['school'] = school_name
        temp_df['filename'] = filename
        
        # setting timestamp and student
        if not temp_df.index.is_integer():
            index_split = list(temp_df.index.str.split(':', 1))           
            temp_df['timestamp'] = np.array(index_split)[:,1]
            temp_df['student'] = np.array(index_split)[:,0]
        else: 
            temp_df.index = temp_df['student'] + ":" + temp_df['timestamp'].astype(str)
        # remove extra columns
        temp_df = temp_df.drop(set(temp_df.columns) - set(column_names), axis=1)
#          temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')] # remove unnamed columns     
        
        # error checking on data
        if not set(column_names).issubset(temp_df.columns):                # check if incomplete columns
            raise Exception('Column names in ' + filename + ' are incomplete')
        if not set(temp_df.columns).issubset(column_names):                # check if has extra columns
            raise Exception('There are extra column names in ' + filename)
        if temp_df['final_act'].isnull().any():                            # check if final act has null
            raise Exception('Null final act in ' + filename)
        if temp_df['final_act'].str.match('VOTE').any():                   # check if final act has 'VOTE' value
            raise Exception('Invalid final act in ' + filename)
    
        label_df = label_df.append(temp_df)

label_df

,a1,a2,filename,final_act,match,person,school,student,text,timestamp,translation
UC-DP01B:13:11:59,InfSoc,InfSoc,UC-DP01A-DP01B.xlsx,InfSoc,True,B,UC,UC-DP01B,hello,13:11:59,hello
UC-DP01A:13:12:01,InfStm,InfStm,UC-DP01A-DP01B.xlsx,InfStm,True,A,UC,UC-DP01A,test,13:12:01,test
UC-DP01B:13:12:06,ResCfm,ResCfm,UC-DP01A-DP01B.xlsx,ResCfm,True,B,UC,UC-DP01B,okay,13:12:06,okay
UC-DP01A:13:12:34,InfNeg,InfNeg,UC-DP01A-DP01B.xlsx,InfNeg,True,A,UC,UC-DP01A,im not good in programming,13:12:34,im not good in programming
UC-DP01B:13:13:29,InfNeg,InfNeg,UC-DP01A-DP01B.xlsx,InfNeg,True,B,UC,UC-DP01B,i dont even know what to do,13:13:29,i dont even know what to do
UC-DP01B:13:14:42,EliOpn,EliOpn,UC-DP01A-DP01B.xlsx,EliOpn,True,B,UC,UC-DP01B,what will we do?,13:14:42,what will we do?
UC-DP01A:13:15:15,ImpAct,ImpAct,UC-DP01A-DP01B.xlsx,ImpAct,True,A,UC,UC-DP01A,just examine what's the erroer and make it cor...,13:15:15,just examine what's the erroer and make it cor...
UC-DP01B:13:15:26,EliOpn,EliOpn,UC-DP01A-DP01B.xlsx,EliOpn,True,B,UC,UC-DP01B,how?,13:15:26,how?
UC-DP01A:13:15:35,ResStm,InfAct,UC-DP01A-DP01B.xlsx,ResStm,False,A,UC,UC-DP01A,by compiling it,13:15:35,by compiling it
UC-DP01B:13:16:19,EliOpn,EliOpn,UC-DP01A-DP01B.xlsx,EliOpn,True,B,UC,UC-DP01B,no what?,13:16:19,no what?


In [32]:
label_df.to_csv(data_proc_dir + 'labelled_data.csv')